In [ ]:
#1

import numpy as np
import pandas as pd
import requests
import math
import statistics
import pprint
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from Secrets.upstox_secrets import UPSTOX_API_KEY #Replace with your own API key
from Secrets.upstox_secrets import UPSTOX_API_SECRET #Replace with your own API secret
from Secrets.upstox_secrets import UPSTOX_CLIENT_ID #Replace with your own client ID

redirect = "https://localhost:3000" #Replace with your own redirect URL
client_id = UPSTOX_CLIENT_ID

url = f"https://api.upstox.com/v2/login/authorization/dialog?response_type=code&client_id={UPSTOX_API_KEY}&redirect_uri={redirect}"
url

In [ ]:
#2

url = "https://api.upstox.com/v2/login/authorization/token"

payload={}
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Accept': 'application/json'
}

code = "AZRrgH" #Replace with your own code

params = {
    "code": code,
    "client_id": f"{UPSTOX_API_KEY}",
    "client_secret": f"{UPSTOX_API_SECRET}",
    "redirect_uri": f"{redirect}",
    "grant_type": "authorization_code"
}

response = requests.post(url, headers=headers, params=params).json()

access_token = response['access_token']

print(access_token)

In [ ]:
#3

#Modify the keys to include your own stocks. You can include any number of stocks.

keys = {"NSE_EQ|INE002A01018": "RELIANCE",
        "NSE_EQ|INE018A01030": "LT",
        "NSE_EQ|INE040A01034": "HDFCBANK",
        "NSE_EQ|INE238A01034": "AXISBANK",
        "NSE_EQ|INE585B01010": "MARUTI",
        "NSE_EQ|INE009A01021": "INFY",
        "NSE_EQ|INE481G01011": "ULTRACEMCO",
        "NSE_EQ|INE397D01024": "BHARTIARTL",
        "NSE_EQ|INE752E01010": "POWERGRID",
        "NSE_EQ|INE758T01015": "ZOMATO",
        "NSE_EQ|INE134E01011": "PFC",
        "NSE_EQ|INE245A01021": "TATAPOWER",
        "NSE_EQ|INE129A01019": "GAIL",
        "NSE_EQ|INE242A01010": "IOC",
        "NSE_EQ|INE003A01024": "SIEMENS",
        "NSE_EQ|INE849A01020": "TRENT",
        "NSE_EQ|INE364U01010": "ADANIGREEN",
        "NSE_EQ|INE259A01022": "COLPAL",
        "NSE_EQ|INE205A01025": "VEDL",
        "NSE_EQ|INE646L01027": "INDIGO",
        "NSE_EQ|INE494B01023": "TVSMOTOR",
        "NSE_EQ|INE318A01026": "PIDILITIND",
        "NSE_EQ|INE102D01028": "GODREJCP",
        "NSE_EQ|INE117A01022": "ABB",
        }

In [ ]:
# 🎛️ ALGORITHM CONFIGURATION - CUSTOMIZE YOUR STRATEGY HERE!
# These parameters control how the algorithm analyzes and trades stocks

# Performance Analysis Timeframes (in days)
TIMEFRAME_1 = 1    # Short-term
TIMEFRAME_2 = 2    # Medium-term
TIMEFRAME_3 = 3   # Long-term

# Performance Score Weightages (must sum to 1.0)
WEIGHT_1 = 0.6     # Weight for TIMEFRAME_1
WEIGHT_2 = 0.3     # Weight for TIMEFRAME_2
WEIGHT_3 = 0.1     # Weight for TIMEFRAME_3

# Rebalancing Frequency (in days)
REBALANCE_FREQUENCY = 3  # How often to rebalance portfolio (default: 7 days)

# Validation
if abs(WEIGHT_1 + WEIGHT_2 + WEIGHT_3 - 1.0) > 0.001:
    print(f"WARNING: Weights don't sum to 1.0! Current sum: {WEIGHT_1 + WEIGHT_2 + WEIGHT_3}")
else:
    print("Configuration validated successfully!")
    
print(f"Timeframes: {TIMEFRAME_1}d ({WEIGHT_1*100:.1f}%), {TIMEFRAME_2}d ({WEIGHT_2*100:.1f}%), {TIMEFRAME_3}d ({WEIGHT_3*100:.1f}%)")
print(f"Rebalancing every {REBALANCE_FREQUENCY} days")


In [ ]:
#4

# CORE ALGORITHM: Analyzes stock performance across multiple timeframes
# This function ranks all stocks based on their momentum scores
def portfolio_custom_date_m1_v3(fund_keys, date):

    data = {'Stock':[], 'Price':[], 'timeframe1':[], 'timeframe2':[], 'timeframe3':[], 'perScore':[], 'analysis':[]}

    open_in = 1
    high_in = 2
    low_in = 3
    close_in = 4

    idx = close_in #Replace with which index you want to use. close_in is the default, which gets the closing price.

    to_date = date.strftime('%Y-%m-%d')
    # Get enough data to cover the longest timeframe + buffer
    
    max_days = max(TIMEFRAME_1, TIMEFRAME_2, TIMEFRAME_3) + 5
    from_date = (date - timedelta(days=max_days)).strftime('%Y-%m-%d')

    for key,value in fund_keys.items():
        
        # V3 API for daily data
        url2 = f'https://api.upstox.com/v3/historical-candle/{key}/days/1/{to_date}/{from_date}'
        headers = {
            'Accept': 'application/json',
            'Authorization': f'Bearer {access_token}'
        }
        response2 = requests.get(url2, headers=headers).json()
        
        # Debug: Check if response has expected structure
        if "data" not in response2:
            print(f"Error for {value}: {response2}")
            continue
        
        if "candles" not in response2["data"]:
            print(f"No candles data for {value}")
            continue

        # Ensure we have sufficient daily data
        candles = response2["data"]["candles"]
        if not candles or len(candles) == 0:
            # print(f"No candle data available for {value}") #Uncomment this to see which stocks have no data
            continue
            
        size = len(candles)
        
        t1 = []    # Timeframe 1 performance
        t2 = []    # Timeframe 2 performance  
        t3 = []    # Timeframe 3 performance
        
        try:
            # MOMENTUM CALCULATION: % change over each timeframe
            # Formula: (current_price - past_price) / past_price * 100
            if size > TIMEFRAME_1:
                a = (candles[0][idx] - candles[TIMEFRAME_1][idx])*100/(candles[TIMEFRAME_1][idx])
                t1.append(a)
            
            # Timeframe 2 performance
            if size > TIMEFRAME_2:
                a = (candles[0][idx] - candles[TIMEFRAME_2][idx])*100/(candles[TIMEFRAME_2][idx])
                t2.append(a)
            
            # Timeframe 3 performance
            if size > TIMEFRAME_3:
                a = (candles[0][idx] - candles[TIMEFRAME_3][idx])*100/(candles[TIMEFRAME_3][idx])
                t3.append(a)

            data['Stock'].append(value)
            data['Price'].append(candles[0][idx])
            data["timeframe1"].append(statistics.mean(t1) if t1 else 0)
            data["timeframe2"].append(statistics.mean(t2) if t2 else 0)
            data["timeframe3"].append(statistics.mean(t3) if t3 else 0)
            
        except (IndexError, ZeroDivisionError, TypeError) as e:
            print(f"Error processing {value}: {e}")
            continue

    # SCORING: Calculate final momentum score for each stock
    for i in range(len(data['Stock'])):
        # Weighted average of all timeframe performances
        score = (data["timeframe1"][i] * WEIGHT_1 + 
                data["timeframe2"][i] * WEIGHT_2 + 
                data["timeframe3"][i] * WEIGHT_3)
        
        if score >= 2:
            data["analysis"].append("buy")
        elif score < 2 and score > 0:
            data["analysis"].append("hold")
        else:
            data["analysis"].append("sell")
        
        data["perScore"].append(score)

    # Sort by performance score
    sort_key = "perScore"
    sort_list = data[sort_key]

    sorted_indices = sorted(range(len(sort_list)), key=lambda k: sort_list[k], reverse=True)

    sorted_stock_data = {key: [value[i] for i in sorted_indices] for key, value in data.items()}

    return sorted_stock_data


In [ ]:
#5

# BACKTESTING ENGINE: Simulates historical trading with the algorithm
class backtest_v3:
    def __init__(self, backtest_time) -> None:   
        # Initialize instance variables with proper values
        self.total_value = 0
        self.old_value = 0
        self.new_value = 0
        self.equal_value = 0
        self.skeys = keys
        self.portfolio_values = []
        self.total_evaluation = []
        
        # Set up the backtest parameters
        self.portfolio_size = float(input("Enter the size of your portfolio: "))
        self.balance = self.portfolio_size
        
        self.s_date = (datetime.now() - timedelta(days = backtest_time*365))
        self.backtest_time = backtest_time*365
        
        # Build initial portfolio
        self.init_portfolio = self.init_build()
        self.old_portfolio = self.init_portfolio.copy() if self.init_portfolio else {}

    def init_build(self):
        # Use V3 function
        retrieved_data = portfolio_custom_date_m1_v3(self.skeys, self.s_date)

        # Ensure we get exactly top 10 stocks
        current_stats = {key: value[:10] for key, value in retrieved_data.items()}
        
        # Verify we have exactly 10 stocks
        if len(current_stats['Stock']) != 10:
            print(f"Warning: Expected 10 stocks, got {len(current_stats['Stock'])}")
        
        position_size = self.portfolio_size / 10  # Always divide by 10 for equal weighting
        self.equal_value = position_size
        
        portfolio = {'Stock': [], 'Holdings': [], 'atPrice': [], 'curPrice':[]}

        # Build portfolio with exactly top 10 stocks
        for i in range(min(10, len(current_stats['Stock']))):
            portfolio['Stock'].append(current_stats['Stock'][i])
            portfolio['atPrice'].append(current_stats['Price'][i])
            portfolio['curPrice'].append(0)

            value = math.floor(position_size/current_stats['Price'][i])

            portfolio['Holdings'].append(value)

            self.total_value += value*current_stats['Price'][i]
            self.balance -= value*current_stats['Price'][i]
        
        return portfolio
    
    def update_port(self):
        self.old_value = self.total_value
        delta = REBALANCE_FREQUENCY
        iterations = int(self.backtest_time/delta)

        file_time = datetime.now().strftime("%Y_%m_%d_%H_%M")
        filename = f"BacktestLogs/test_backtest_log_{file_time}.txt"
        csv_filename = f"BacktestLogs/backtest_detailed_{file_time}.csv"

        # Initialize comprehensive logging data
        all_transactions = []
        portfolio_changes = []
        performance_summary = []

        with open(filename, "w") as file:
            file.write("="*80 + "\n")
            file.write(f"📊 ALGORITHMIC TRADING BACKTEST REPORT\n")
            file.write(f"Period: {self.s_date.strftime('%Y-%m-%d')} to {(self.s_date + timedelta(days=self.backtest_time)).strftime('%Y-%m-%d')}\n")
            file.write(f"Initial Portfolio Size: ₹{self.portfolio_size:,.2f}\n")
            file.write("="*80 + "\n\n")

        for iteration in tqdm(range(iterations), desc="Backtesting Progress"):
            current_date = self.s_date + timedelta(days=delta)
            week_number = iteration + 1
            
            time.sleep(6) #IMPORTANT: Read Below
            '''
            This is to avoid rate limiting. DO NOT CHANGE THIS.
            If you use a different API, change this to the appropriate
            value after reading the API documentation.
            '''

            # Get current top performers using V3 function
            total_retrieved = portfolio_custom_date_m1_v3(self.skeys, current_date)
            top_10_performers = {key: value[:10] for key, value in total_retrieved.items()}

            # Store old portfolio for comparison
            old_portfolio_stocks = self.init_portfolio['Stock'].copy()

            # Calculate current portfolio value before rebalancing
            current_portfolio_value = 0
            for i in range(len(self.init_portfolio['Stock'])):
                try:
                    stock_idx = total_retrieved['Stock'].index(self.init_portfolio['Stock'][i])
                    current_price = total_retrieved['Price'][stock_idx]
                    self.init_portfolio['curPrice'][i] = current_price
                    current_portfolio_value += self.init_portfolio['Holdings'][i] * current_price
                except ValueError:
                    current_portfolio_value += self.init_portfolio['Holdings'][i] * self.init_portfolio['curPrice'][i]

            total_capital = current_portfolio_value + self.balance

            # Create detailed transaction tables
            sell_transactions = []
            buy_transactions = []
            dropped_stocks = []
            
            # SMART REBALANCING: Only trade when necessary
            current_top_10 = top_10_performers['Stock'][:10]  # Ensure exactly 10
            stocks_to_drop = [stock for stock in old_portfolio_stocks if stock not in current_top_10]
            stocks_to_add = [stock for stock in current_top_10 if stock not in old_portfolio_stocks]
            stocks_to_keep = [stock for stock in old_portfolio_stocks if stock in current_top_10]

            # SELL TRANSACTIONS - Only sell stocks being dropped
            for i in range(len(self.init_portfolio['Stock'])):
                stock_name = self.init_portfolio['Stock'][i]
                if stock_name in stocks_to_drop and self.init_portfolio['Holdings'][i] > 0:
                    holdings = self.init_portfolio['Holdings'][i]
                    
                    try:
                        stock_idx = total_retrieved['Stock'].index(stock_name)
                        sell_price = total_retrieved['Price'][stock_idx]
                        current_score = total_retrieved['perScore'][stock_idx]
                        current_rank = stock_idx + 1
                    except ValueError:
                        sell_price = self.init_portfolio['curPrice'][i]
                        current_score = 0
                        current_rank = "N/A"
                    
                    sell_value = holdings * sell_price
                    self.balance += sell_value
                    
                    sell_transactions.append({
                        'Date': current_date.strftime('%Y-%m-%d'),
                        'Week': week_number,
                        'Action': 'SELL',
                        'Stock': stock_name,
                        'Shares': holdings,
                        'Price': sell_price,
                        'Total_Value': sell_value,
                        'Performance_Score': current_score,
                        'Current_Rank': current_rank
                    })

                    dropped_stocks.append({
                        'Date': current_date.strftime('%Y-%m-%d'),
                        'Week': week_number,
                        'Stock': stock_name,
                        'Shares_Sold': holdings,
                        'Sell_Price': sell_price,
                        'Sell_Value': sell_value,
                        'Performance_Score': current_score,
                        'Reason': 'Dropped from Top 10'
                    })

            # Update portfolio - remove dropped stocks, keep existing ones, prepare for new ones
            new_portfolio = {'Stock': [], 'Holdings': [], 'atPrice': [], 'curPrice': []}
            
            # Keep existing stocks that are still in top 10
            for i in range(len(self.init_portfolio['Stock'])):
                stock_name = self.init_portfolio['Stock'][i]
                if stock_name in stocks_to_keep:
                    # Update current price but keep holdings
                    try:
                        stock_idx = total_retrieved['Stock'].index(stock_name)
                        current_price = total_retrieved['Price'][stock_idx]
                    except ValueError:
                        current_price = self.init_portfolio['curPrice'][i]
                    
                    new_portfolio['Stock'].append(stock_name)
                    new_portfolio['Holdings'].append(self.init_portfolio['Holdings'][i])
                    new_portfolio['atPrice'].append(self.init_portfolio['atPrice'][i])
                    new_portfolio['curPrice'].append(current_price)

            # Calculate equal allocation for new stocks only
            equal_allocation = total_capital / 10
            
            # BUY TRANSACTIONS - Only buy new stocks
            for stock_name in stocks_to_add:
                stock_idx = current_top_10.index(stock_name)
                stock_price = top_10_performers['Price'][stock_idx]
                performance_score = top_10_performers['perScore'][stock_idx]
                
                shares_to_buy = math.floor(equal_allocation / stock_price)
                actual_investment = shares_to_buy * stock_price
                
                # Add new stock to portfolio
                new_portfolio['Stock'].append(stock_name)
                new_portfolio['Holdings'].append(shares_to_buy)
                new_portfolio['atPrice'].append(stock_price)
                new_portfolio['curPrice'].append(stock_price)
                
                self.balance -= actual_investment
                
                buy_transactions.append({
                    'Date': current_date.strftime('%Y-%m-%d'),
                    'Week': week_number,
                    'Action': 'BUY',
                    'Stock': stock_name,
                    'Shares': shares_to_buy,
                    'Price': stock_price,
                    'Total_Value': actual_investment,
                    'Performance_Score': performance_score,
                    'Rank': stock_idx + 1,
                    'New_Stock': 'Yes'
                })

            # Update the portfolio
            self.init_portfolio = new_portfolio

            # Calculate new portfolio value
            new_portfolio_value = 0
            for i in range(len(self.init_portfolio['Stock'])):
                new_portfolio_value += self.init_portfolio['Holdings'][i] * self.init_portfolio['curPrice'][i]
            
            self.portfolio_values.append(new_portfolio_value)
            total_evaluation = new_portfolio_value + self.balance
            self.total_evaluation.append(total_evaluation)

            # Store all transactions
            all_transactions.extend(sell_transactions)
            all_transactions.extend(buy_transactions)

            # Create portfolio change summary
            portfolio_changes.append({
                'Date': current_date.strftime('%Y-%m-%d'),
                'Week': week_number,
                'Portfolio_Value_Before': round(current_portfolio_value, 2),
                'Cash_Before': round(self.balance + sum([t['Total_Value'] for t in buy_transactions]), 2),
                'Total_Capital': round(total_capital, 2),
                'Portfolio_Value_After': round(new_portfolio_value, 2),
                'Cash_After': round(self.balance, 2),
                'Total_Evaluation': round(total_evaluation, 2),
                'Stocks_Dropped': len(dropped_stocks),
                'New_Stocks_Added': len([t for t in buy_transactions if t['New_Stock'] == 'Yes'])
            })

            # Write formatted tables to text file
            with open(filename, "a") as file:
                file.write(f"\n{'='*80}\n")
                file.write(f"📅 WEEK {week_number} - {current_date.strftime('%Y-%m-%d')}\n")
                file.write(f"{'='*80}\n\n")

                # Summary table
                file.write("📊 REBALANCING SUMMARY\n")
                file.write("-" * 50 + "\n")
                file.write(f"Portfolio Value Before: ₹{current_portfolio_value:>15,.2f}\n")
                file.write(f"Cash Balance Before:    ₹{self.balance + sum([t['Total_Value'] for t in buy_transactions]):>15,.2f}\n")
                file.write(f"Total Capital:          ₹{total_capital:>15,.2f}\n")
                file.write(f"Stocks Kept:            {len(stocks_to_keep):>15}\n")
                file.write(f"Stocks Dropped:         {len(stocks_to_drop):>15}\n")
                file.write(f"New Stocks Added:       {len(stocks_to_add):>15}\n")
                file.write(f"Portfolio Value After:  ₹{new_portfolio_value:>15,.2f}\n")
                file.write(f"Cash Balance After:     ₹{self.balance:>15,.2f}\n")
                file.write(f"Total Evaluation:       ₹{total_evaluation:>15,.2f}\n")
                file.write("-" * 50 + "\n\n")
                
                # Show stocks being kept (no transactions)
                if stocks_to_keep:
                    file.write("🔄 STOCKS KEPT IN PORTFOLIO\n")
                    file.write("-" * 60 + "\n")
                    file.write(f"{'Stock':<12} {'Holdings':<10} {'Current Price':<15} {'Value':<12}\n")
                    file.write("-" * 60 + "\n")
                    for stock_name in stocks_to_keep:
                        for i, portfolio_stock in enumerate(self.init_portfolio['Stock']):
                            if portfolio_stock == stock_name:
                                holdings = self.init_portfolio['Holdings'][i]
                                price = self.init_portfolio['curPrice'][i]
                                value = holdings * price
                                file.write(f"{stock_name:<12} {holdings:<10} ₹{price:<14.2f} ₹{value:<11.2f}\n")
                                break
                    file.write("-" * 60 + "\n\n")

                # Sell transactions table
                if sell_transactions:
                    file.write("🔴 SELL TRANSACTIONS\n")
                    file.write("-" * 80 + "\n")
                    file.write(f"{'Stock':<12} {'Shares':<8} {'Price':<10} {'Value':<12} {'Score':<8} {'Rank':<6}\n")
                    file.write("-" * 80 + "\n")
                    for t in sell_transactions:
                        file.write(f"{t['Stock']:<12} {t['Shares']:<8} ₹{t['Price']:<9.2f} ₹{t['Total_Value']:<11.2f} {t['Performance_Score']:<8.2f} {t['Current_Rank']:<6}\n")
                    file.write("-" * 80 + "\n\n")

                # Buy transactions table
                if buy_transactions:
                    file.write("🟢 BUY TRANSACTIONS (New Stocks Only)\n")
                    file.write("-" * 80 + "\n")
                    file.write(f"{'Stock':<12} {'Shares':<8} {'Price':<10} {'Value':<12} {'Score':<8} {'Rank':<6}\n")
                    file.write("-" * 80 + "\n")
                    for t in buy_transactions:
                        file.write(f"{t['Stock']:<12} {t['Shares']:<8} ₹{t['Price']:<9.2f} ₹{t['Total_Value']:<11.2f} {t['Performance_Score']:<8.2f} {t['Rank']:<6}\n")
                    file.write("-" * 80 + "\n\n")

                # Dropped stocks table
                if dropped_stocks:
                    file.write("❌ STOCKS DROPPED FROM PORTFOLIO\n")
                    file.write("-" * 80 + "\n")
                    file.write(f"{'Stock':<12} {'Shares':<8} {'Price':<10} {'Value':<12} {'Score':<8} {'Reason':<15}\n")
                    file.write("-" * 80 + "\n")
                    for d in dropped_stocks:
                        file.write(f"{d['Stock']:<12} {d['Shares_Sold']:<8} ₹{d['Sell_Price']:<9.2f} ₹{d['Sell_Value']:<11.2f} {d['Performance_Score']:<8.2f} {d['Reason']:<15}\n")
                    file.write("-" * 80 + "\n\n")

            delta += REBALANCE_FREQUENCY

        # Save comprehensive CSV file
        if all_transactions:
            df_transactions = pd.DataFrame(all_transactions)
            df_transactions.to_csv(csv_filename, index=False)

        # Save portfolio changes CSV
        if portfolio_changes:
            df_changes = pd.DataFrame(portfolio_changes)
            df_changes.to_csv(f"BacktestLogs/portfolio_changes_{file_time}.csv", index=False)

        # Calculate final portfolio value
        self.new_value = 0
        for i in range(len(self.init_portfolio['Stock'])):
            self.new_value += self.init_portfolio['Holdings'][i] * self.init_portfolio['curPrice'][i]

        # Write final summary
        with open(filename, "a") as file:
            file.write(f"\n{'='*80}\n")
            file.write("📈 FINAL BACKTEST SUMMARY\n")
            file.write(f"{'='*80}\n")
            file.write(f"Initial Investment:     ₹{self.old_value:>15,.2f}\n")
            file.write(f"Final Portfolio Value:  ₹{self.new_value:>15,.2f}\n")
            file.write(f"Final Cash Balance:     ₹{self.balance:>15,.2f}\n")
            file.write(f"Total Final Value:      ₹{self.new_value + self.balance:>15,.2f}\n")
            file.write(f"Total Return:           ₹{(self.new_value + self.balance) - self.old_value:>15,.2f}\n")
            file.write(f"Return Percentage:      {((self.new_value + self.balance) / self.old_value - 1) * 100:>15.2f}%\n")
            file.write(f"{'='*80}\n")

        print(f"\n✅ Detailed backtest logs saved:")
        print(f"📄 Text Report: {filename}")
        print(f"📊 Transaction Data: {csv_filename}")
        print(f"📈 Portfolio Changes: BacktestLogs/portfolio_changes_{file_time}.csv")
    
    def old_graph(self):
        portfolio = self.old_portfolio
        initial_value = 0

        for i in range(len(portfolio['Stock'])):
            initial_value += portfolio['Holdings'][i]*portfolio['atPrice'][i]

        cmap = plt.get_cmap('viridis')

        fig, ax = plt.subplots()
        wedges, texts = ax.pie(portfolio['Holdings'], autopct=None, startangle=140, colors=cmap(np.linspace(0, 1, len(portfolio['Stock']))))
        ax.axis('equal')

        legend_labels = [f'{label}: {size}' for label, size in zip(portfolio['Stock'], portfolio['Holdings'])]
        ax.legend(wedges, legend_labels, title="Categories", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
        plt.title(f'Portfolio Details, Initial Investment: {initial_value:.2f}')
        plt.show()

    def new_graph(self):
        portfolio = self.init_portfolio
        initial_value = 0

        for i in range(len(portfolio['Stock'])):
            initial_value += portfolio['Holdings'][i]*portfolio['curPrice'][i]

        cmap = plt.get_cmap('viridis')

        fig, ax = plt.subplots()
        wedges, texts = ax.pie(portfolio['Holdings'], autopct=None, startangle=140, colors=cmap(np.linspace(0, 1, len(portfolio['Stock']))))
        ax.axis('equal')

        legend_labels = [f'{label}: {size}' for label, size in zip(portfolio['Stock'], portfolio['Holdings'])]
        ax.legend(wedges, legend_labels, title="Categories", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
        plt.title(f'Portfolio Details, Current Valuation: {initial_value:.2f}')
        plt.show()

    def portfolio_growth(self):
        y_values = self.portfolio_values
        x_values = np.arange(0, len(self.portfolio_values))

        plt.figure(figsize=(12, 8))
        plt.plot(x_values, y_values, label='Portfolio Growth Chart', color='red')
        plt.xlabel('Time Frame (weeks from the start)')
        plt.ylabel('Portfolio Value (Rs)')
        plt.title('Portfolio Growth Chart')
        plt.legend()
        plt.show()

    def evaluation(self):
        y_values = self.total_evaluation
        x_values = np.arange(0, len(self.total_evaluation))

        plt.figure(figsize=(12, 8))
        plt.plot(x_values, y_values, label='Total Evaluation Growth', color='green')
        plt.xlabel('Time Frame (weeks from the start)')
        plt.ylabel('Evaluation (Rs)')
        plt.title('Evaluation Growth Chart')
        plt.legend()
        plt.show()

    def combined_growth_chart(self):
        """
        VISUALIZATION: Shows portfolio performance over time
        Blue = Stock holdings value, Green = Total value (stocks + cash)
        """
        # Prepare data
        portfolio_values = self.portfolio_values
        evaluation_values = self.total_evaluation
        x_values = np.arange(0, len(portfolio_values))
        
        # Create figure with larger size for better visibility
        plt.figure(figsize=(14, 10))
        
        # Plot both lines
        plt.plot(x_values, portfolio_values, label='Portfolio Growth', color='blue', linewidth=2)
        plt.plot(x_values, evaluation_values, label='Total Evaluation', color='green', linewidth=2)
        
        # Customize the plot
        plt.xlabel('Time Frame (weeks from the start)', fontsize=12)
        plt.ylabel('Value (₹)', fontsize=12)
        plt.title('Portfolio vs Total Evaluation Growth Comparison', fontsize=14, fontweight='bold')
        
        # Add grid for better readability
        plt.grid(True, alpha=0.3)
        
        # Customize legend
        plt.legend(fontsize=11, loc='upper left')
        
        # Format y-axis to show values in a readable format
        plt.ticklabel_format(style='plain', axis='y')
        
        # Add some padding to the plot
        plt.tight_layout()
        
        # Show the plot
        plt.show()
        
        # Print summary statistics
        if len(portfolio_values) > 0 and len(evaluation_values) > 0:
            print(f"\n📊 Growth Summary:")
            print(f"Portfolio Growth: ₹{portfolio_values[0]:.2f} → ₹{portfolio_values[-1]:.2f}")
            print(f"Total Evaluation: ₹{evaluation_values[0]:.2f} → ₹{evaluation_values[-1]:.2f}")
            print(f"Cash Balance: ₹{self.balance:.2f}")

    def calculate_CAGR(self):
        Vf = self.new_value + self.balance
        Vi = self.old_value
        n = math.ceil(self.backtest_time/365)

        print(f"Final Value: {Vf:.2f}")
        print(f"Initial Value: {Vi:.2f}")
        print("Time (years): ", n)

        if Vi > 0:
            cagr = (((Vf/Vi)**(1/n)) - 1)*100
            return (f"CAGR: {cagr:.2f}%")
        else:
            return "Cannot calculate CAGR: Initial value is zero"

# Test the modular configuration
print("Current Algorithm Configuration:")
print(f"Analysis Timeframes: {TIMEFRAME_1} days, {TIMEFRAME_2} days, {TIMEFRAME_3} days")
print(f"Performance Weights: {WEIGHT_1}, {WEIGHT_2}, {WEIGHT_3}")
print(f"Rebalancing Frequency: Every {REBALANCE_FREQUENCY} days")

In [ ]:
# TEST: See current top-performing stocks
print("Analyzing current market...")
test = portfolio_custom_date_m1_v3(keys, datetime.now())
print("\nCurrent top 10 stocks by momentum score:")
for i, stock in enumerate(test['Stock'][:10], 1):
    score = test['perScore'][test['Stock'].index(stock)]
    print(f"{i:2d}. {stock:<12} (Score: {score:6.2f})")
print(f"\nAlgorithm would select these {min(10, len(test['Stock']))} stocks for the portfolio")

In [ ]:
# START BACKTEST: This will simulate historical trading
# Recommended: Start with 1-2 years for faster testing
backtest_time = int(input("Enter backtest duration (Years, maximum 10): "))
test = backtest_v3(backtest_time)

In [ ]:
print(test.old_portfolio)
test.old_graph() #Gets the initial portfolio graph

In [ ]:
test.update_port() #Run the backtest, this will take a while.

In [ ]:
test.new_graph() #Gets the final portfolio graph

In [ ]:
print(test.calculate_CAGR())
print(f"Balance: {test.balance:.2f}")

In [ ]:
test.evaluation()

In [ ]:
test.portfolio_growth()

In [ ]:
#Combined growth chart (Portfolio and Evaluation)
test.combined_growth_chart()